In [ ]:
!pip install pandas numpy
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#updated
import math
import random
import pandas as pd
from typing import List
import time
from google.colab import files

START_TIME = 8 * 60
END_TIME = 20 * 60
COURIER_SPEED = 15.0
EARTH_RADIUS = 6378.137

class Location:
    def __init__(self, id, lng, lat, location_type):
        self.id, self.lng, self.lat, self.type = id, lng, lat, location_type

class Order:
    def __init__(self, order_id, spot_id, num_packages):
        self.id, self.spot_id, self.num_packages = order_id, spot_id, num_packages

class EcommerceOrder(Order):
    def __init__(self, order_id, spot_id, site_id, num_packages):
        super().__init__(order_id, spot_id, num_packages)
        self.site_id = site_id

class O2OOrder(Order):
    def __init__(self, order_id, spot_id, shop_id, pickup_time, delivery_time, num_packages):
        super().__init__(order_id, spot_id, num_packages)
        self.shop_id, self.pickup_time, self.delivery_time = shop_id, pickup_time, delivery_time

class CourierRoute:
    def __init__(self, courier_id):
        self.courier_id, self.route, self.current_load = courier_id, [], 0

class DeliveryProblem:
    def __init__(self):
        self.branches, self.pois, self.shops, self.orders, self.couriers, self.distance_cache = {}, {}, {}, [], [], {}

    def calculate_distance(self, loc1, loc2):
        key = tuple(sorted([loc1.id, loc2.id]))
        if key in self.distance_cache:
            return self.distance_cache[key]
        lat1, lng1, lat2, lng2 = map(math.radians, [loc1.lat, loc1.lng, loc2.lat, loc2.lng])
        dlat, dlng = (lat1 - lat2) / 2, (lng1 - lng2) / 2
        distance = 2 * EARTH_RADIUS * math.asin(math.sqrt(math.sin(dlat)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlng)**2))
        self.distance_cache[key] = distance
        return distance

    def calculate_travel_time(self, loc1, loc2):
        return self.calculate_distance(loc1, loc2) / COURIER_SPEED * 60

    def calculate_processing_time(self, num_packages):
        return 3 * math.sqrt(num_packages) + 5

    def convert_time_to_minutes(self, time_str):
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes

def load_problem_data(problem):
    for _, row in pd.read_csv("/content/branches.csv").iterrows():
        problem.branches[row['Site_id']] = Location(row['Site_id'], row['Lng'], row['Lat'], 'branch')
    for _, row in pd.read_csv("/content/pois.csv").iterrows():
        problem.pois[row['Spot_id']] = Location(row['Spot_id'], row['Lng'], row['Lat'], 'poi')
    for _, row in pd.read_csv("/content/shops.csv").iterrows():
        problem.shops[row['Shop_id']] = Location(row['Shop_id'], row['Lng'], row['Lat'], 'shop')
    for _, row in pd.read_csv("/content/ecommerce_orders.csv").iterrows():
        problem.orders.append(EcommerceOrder(row['Order_id'], row['Spot_id'], row['Site_id'], row['Num']))
    for _, row in pd.read_csv("/content/o2o_orders.csv").iterrows():
        pickup = problem.convert_time_to_minutes(row['Pickup_time'])
        delivery = problem.convert_time_to_minutes(row['Delivery_time'])
        problem.orders.append(O2OOrder(row['Order_id'], row['Spot_id'], row['Shop_id'], pickup, delivery, row['Num']))
    for courier_id in pd.read_csv("/content/couriers.csv")['Courier_id']:
        problem.couriers.append(CourierRoute(courier_id))

def create_initial_solution(problem):
    couriers, orders = problem.couriers, problem.orders.copy()
    random.shuffle(orders)
    for i, order in enumerate(orders):
        couriers[i % len(couriers)].route.append(order)
    return couriers

def tabu_search(problem, initial_solution, max_iterations=1000, tabu_size=20):
    best = current = initial_solution
    tabu = []

    def solution_cost(solution):
        total = 0
        for route in solution:
            time, prev = 0, None
            for order in route.route:
                loc = problem.pois.get(order.spot_id) or problem.shops.get(order.spot_id) or problem.branches.get(order.spot_id)
                if prev:
                    time += problem.calculate_travel_time(prev, loc)
                time += problem.calculate_processing_time(order.num_packages)
                prev = loc
            total += time
        return total

    best_cost = solution_cost(best)

    for _ in range(max_iterations):
        neighborhood = []
        for _ in range(10):
            new = [CourierRoute(r.courier_id) for r in current]
            for idx, r in enumerate(current): new[idx].route = r.route.copy()
            r1, r2 = random.sample(range(len(new)), 2)
            if new[r1].route and new[r2].route:
                i1, i2 = random.randint(0, len(new[r1].route)-1), random.randint(0, len(new[r2].route)-1)
                new[r1].route[i1], new[r2].route[i2] = new[r2].route[i2], new[r1].route[i1]
            neighborhood.append(new)

        neighborhood.sort(key=solution_cost)
        current = neighborhood[0]
        current_cost = solution_cost(current)
        if current_cost < best_cost:
            best, best_cost = current, current_cost

        tabu.append(current)
        if len(tabu) > tabu_size: tabu.pop(0)

    return best

problem = DeliveryProblem()
load_problem_data(problem)
initial_solution = create_initial_solution(problem)
final_solution = tabu_search(problem, initial_solution)

solution_data = []
for route in final_solution:
    current_time, prev_addr = START_TIME, None
    for order in route.route:
        pickup_addr = getattr(order, 'site_id', None) or getattr(order, 'shop_id', None)
        delivery_addr = order.spot_id

        if prev_addr:
            current_time += problem.calculate_travel_time(problem.pois[prev_addr], problem.branches.get(pickup_addr, problem.shops.get(pickup_addr)))
        arrival_time_pickup = current_time
        departure_pickup = arrival_time_pickup + problem.calculate_processing_time(order.num_packages)
        solution_data.append({"Courier_id": route.courier_id, "Addr": pickup_addr, "Arrival_time": arrival_time_pickup, "Departure": departure_pickup, "Amount": order.num_packages, "Order_id": order.id})

        current_time = departure_pickup + problem.calculate_travel_time(problem.branches.get(pickup_addr, problem.shops.get(pickup_addr)), problem.pois[delivery_addr])
        arrival_time_delivery = current_time
        departure_delivery = arrival_time_delivery + problem.calculate_processing_time(order.num_packages)
        solution_data.append({"Courier_id": route.courier_id, "Addr": delivery_addr, "Arrival_time": arrival_time_delivery, "Departure": departure_delivery, "Amount": -order.num_packages, "Order_id": order.id})
        prev_addr = delivery_addr
        current_time = departure_delivery

solution_df = pd.DataFrame(solution_data)
solution_df.to_csv('solution_new.csv', index=False)
files.download('solution_new.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Calculate total couriers used
total_couriers_used = len([route for route in final_solution if route.route])
print(f"Total couriers used: {total_couriers_used}")

# Calculate total cost
total_cost = 0
for route in final_solution:
    route_cost, prev_addr = 0, None
    current_time = START_TIME
    for order in route.route:
        pickup_addr = getattr(order, 'site_id', None) or getattr(order, 'shop_id', None)
        delivery_addr = order.spot_id

        # Pickup step
        if prev_addr:
            current_time += problem.calculate_travel_time(problem.pois[prev_addr], problem.branches.get(pickup_addr, problem.shops.get(pickup_addr)))
        current_time += problem.calculate_processing_time(order.num_packages)

        # Delivery step
        current_time += problem.calculate_travel_time(problem.branches.get(pickup_addr, problem.shops.get(pickup_addr)), problem.pois[delivery_addr])
        current_time += problem.calculate_processing_time(order.num_packages)

        prev_addr = delivery_addr

    route_cost = current_time - START_TIME
    total_cost += route_cost

print(f"Total cost of the solution: {total_cost:.0f}")


Total couriers used: 1000
Total cost of the solution: 1892390


In [ ]:
import pandas as pd
import numpy as np

sub_df = pd.read_csv("solution_new.csv")






import pandas as pd
import numpy as np

sub_df = pd.read_csv("solution_new.csv", names=["Courier_id", "Addr", "Arrival_time", "Departure", "Amount", "Order_id"])

SPEED = 15 * 1000.0 / 60
site_df = pd.read_csv("/content/branches.csv")
spot_df = pd.read_csv("/content/pois.csv")
shop_df = pd.read_csv("/content/shops.csv")
ds_order_df = pd.read_csv("/content/ecommerce_orders.csv")
o2o_order_df = pd.read_csv("/content/o2o_orders.csv")
node_lng_dict = {}
node_lat_dict = {}
for i, e in site_df.iterrows():
    node_lng_dict[e.Site_id] = e.Lng
    node_lat_dict[e.Site_id] = e.Lat

for i, e in spot_df.iterrows():
    node_lng_dict[e.Spot_id] = e.Lng
    node_lat_dict[e.Spot_id] = e.Lat

for i, e in shop_df.iterrows():
    node_lng_dict[e.Shop_id] = e.Lng
    node_lat_dict[e.Shop_id] = e.Lat

def time2min(time):
    ts = pd.to_datetime(time, format="%H:%M")
    return ts.hour*60+ts.minute-480

def cal_distance(node1, node2):
    global node_lng_dict, node_lat_dict
    lng1 = node_lng_dict[node1]
    lat1 = node_lat_dict[node1]
    lng2 = node_lng_dict[node2]
    lat2 = node_lat_dict[node2]
    R = 6378137.0
    d_lat = (lat1-lat2)/2.0
    d_lng = (lng1-lng2)/2.0
    S = 2*R*np.arcsin(np.sqrt(np.power(np.sin(np.pi/180*d_lat),2)+np.cos(np.pi/180*lat1)*np.cos(np.pi/180*lat2)*np.power(np.sin(np.pi/180*d_lng),2)))
    return S

def cal_cost(node1, node2):
    global SPEED
    return np.round(cal_distance(node1, node2)/SPEED)

def cal_proctime(x):
    return np.round(3*np.sqrt(x)+5)

arranges = []
for i in range(1, 1001):
    arrange = sub_df[sub_df.Courier_id==("D%04d"%i)]
    if len(arrange)==0:
        continue
    arranges.append(arrange)

def verify2(arrange):
    departure = arrange.iloc[0].Departure
    for i in range(1, len(arrange)):
        arrival_time = arrange.iloc[i].Arrival_time
        if arrival_time < departure:
            return False
        departure = arrange.iloc[i].Departure
    return True

def verify3(arrange):
    for i in range(0, len(arrange)):
        arrival_time = arrange.iloc[i].Arrival_time
        departure = arrange.iloc[i].Departure
        if departure < arrival_time:
            return False
    return True

def verify9(arrange, ds_order_df, o2o_order_df):
    # use the example to verify cal functions
    #mix_path = ["A083","A083","A083","A083","A083","B5800","B7555","B7182","B8307","B8461","A083","A083","A083","B6528","S245","B3266","B3266","B2337","A083","A083","A083","A083","S294","B1940","B6104","B8926","B9072","B6103"]
    #orders = ["F6344","F6360","F6358","F6353","F6354","F6344","F6354","F6353","F6358","F6360","F6349","F6325","F6314","F6349","E0895","E0895","F6325","F6314","F6366","F6345","F6346","F6308","E1088","F6308","F6346","E1088","F6366","F6345"]
    mix_path = arrange.Addr.tolist()
    orders = arrange.Order_id.tolist()
    mix_arrange_df = pd.DataFrame({ "Addr": mix_path, "Order_id": orders})
    mix_arrange_df["Courier_id"] = arrange.iloc[0].Courier_id
    mix_arrange_df["Arrival_time"] = arrange.iloc[0].Arrival_time
    mix_arrange_df["Departure"] = arrange.iloc[0].Departure
    mix_arrange_df["Amount"] = 0
    pre_e = mix_arrange_df.iloc[0]
    pre_addr = pre_e.Addr
    pre_order = pre_e.Order_id
    pre_departure = arrange.iloc[0].Departure
    pre_departure = float(pre_departure)  # Ensure it's a number

    if 'F' in pre_order:
        amount = ds_order_df[ds_order_df["Order_id"]==pre_order].Num.values[0]
    elif 'E' in pre_order:
        amount = o2o_order_df[o2o_order_df["Order_id"]==pre_order].Num.values[0]
    new_mix_arrange_df = pd.DataFrame({"Courier_id": [arrange.iloc[0].Courier_id], "Addr": [pre_addr], "Arrival_time": [arrange.iloc[0].Arrival_time], "Departure": [arrange.iloc[0].Departure], "Amount": [amount],  "Order_id": [pre_order]})
    for i in range(1, len(mix_arrange_df)):
        cur_e = mix_arrange_df.iloc[i]
        cur_addr = cur_e.Addr
        cur_order = cur_e.Order_id
        if 'A' in cur_addr:
            arrival_time = pre_departure + cal_cost(pre_addr, cur_addr)
            amount = ds_order_df[ds_order_df["Order_id"]==cur_order].Num.values[0]
            departure = arrival_time
        elif 'B' in cur_addr and 'F' in cur_order:
            arrival_time = pre_departure + cal_cost(pre_addr, cur_addr)
            amount = -ds_order_df[ds_order_df["Order_id"]==cur_order].Num.values[0]
            departure = arrival_time + cal_proctime(abs(amount))
        elif 'B' in cur_addr and 'E' in cur_order:
            arrival_time = pre_departure + cal_cost(pre_addr, cur_addr)
            amount = -o2o_order_df[o2o_order_df["Order_id"]==cur_order].Num.values[0]
            departure = arrival_time + cal_proctime(abs(amount))
        elif 'S' in cur_addr:
            arrival_time = pre_departure + cal_cost(pre_addr, cur_addr)
            amount = o2o_order_df[o2o_order_df["Order_id"]==cur_order].Num.values[0]
            departure = max(time2min(o2o_order_df[o2o_order_df["Order_id"]==cur_order].Pickup_time.values[0]), arrival_time)
        new_row = pd.DataFrame({"Courier_id": [arrange.iloc[0].Courier_id], "Addr": [cur_addr], "Arrival_time": [arrival_time], "Departure": [departure], "Amount": [amount],  "Order_id": [cur_order]})
        new_mix_arrange_df = pd.concat([new_mix_arrange_df, new_row], ignore_index=True)
        cmp_e = arrange.iloc[i]
        if cur_addr != cmp_e.Addr or amount != cmp_e.Amount or arrival_time != cmp_e.Arrival_time or departure != cmp_e.Departure:
            # print("arrange {}".format(i))
            return False
        pre_addr = cur_addr
        pre_departure = departure
    return True


def verify_v5(sub_df, o2o_order_df, ds_order_df):
    sub_df_sort = sub_df.sort_values(["Order_id", "Departure"], ascending=[1, 1])
    for i in range(0, len(o2o_order_df)*2, 2):
        shop_id = sub_df_sort.iloc[i].Addr
        shop_amount = sub_df_sort.iloc[i].Amount
        spot_id = sub_df_sort.iloc[i+1].Addr
        spot_amount = sub_df_sort.iloc[i+1].Amount
        if shop_id != o2o_order_df.iloc[i//2].Shop_id:
            print("o2o shop_id {}".format(i))
            return False
        if spot_id != o2o_order_df.iloc[i//2].Spot_id:
            print("o2o spot_id {}".format(i))
            return False
        if shop_amount != o2o_order_df.iloc[i//2].Num:
            print("o2o shop_amount {}".format(i))
            return False
        if spot_amount != -o2o_order_df.iloc[i//2].Num:
            print("o2o spot_amount {}".format(i))
            return False
    for i in range(len(o2o_order_df)*2, len(o2o_order_df)*2+len(ds_order_df)*2, 2):
        site_id = sub_df_sort.iloc[i].Addr
        site_amount = sub_df_sort.iloc[i].Amount
        spot_id = sub_df_sort.iloc[i+1].Addr
        spot_amount = sub_df_sort.iloc[i+1].Amount
        if site_id != ds_order_df.iloc[i//2-len(o2o_order_df)].Site_id:
            print("ds site_id {}".format(i))
            return False
        if spot_id != ds_order_df.iloc[i//2-len(o2o_order_df)].Spot_id:
            print("ds spot_id {}".format(i))
            return False
        if site_amount != ds_order_df.iloc[i//2-len(o2o_order_df)].Num:
            print("ds site_amount {}".format(i))
            return False
        if spot_amount != -ds_order_df.iloc[i//2-len(o2o_order_df)].Num:
            print("ds spot_amount {}".format(i))
            return False
    return True

def verify_v6(sub_df, o2o_order_df):
    sub_df_sort = sub_df.sort_values(["Order_id", "Departure"], ascending=[1, 1])
    for i in range(0, len(o2o_order_df)*2, 2):
        shop_departure = sub_df_sort.iloc[i].Departure
        # Convert shop_departure to int if it's a string
        if isinstance(shop_departure, str):
            try:
                shop_departure = int(shop_departure)
            except ValueError:
                print(f"Warning: Could not convert shop_departure '{shop_departure}' to integer")
                return False

        pickup_time = o2o_order_df.iloc[i//2].Pickup_time
        pickup_min = time2min(pickup_time)

        if shop_departure < pickup_min:
            print("shop_departure {}".format(i))
            return False
    return True

def verify_v7(sub_df, o2o_order_df, ds_order_df):
    sub_df_sort = sub_df.sort_values(["Order_id", "Departure"], ascending=[1, 1])
    for i in range(0, len(o2o_order_df) * 2, 2):
        if i + 1 >= len(sub_df_sort):  # Boundary check
            break
        shop_num = sub_df_sort.iloc[i].Amount
        spot_num = sub_df_sort.iloc[i + 1].Amount
        if shop_num + spot_num != 0:
            print("shop_num {}".format(i))
            return False

    for i in range(len(o2o_order_df) * 2, len(o2o_order_df) * 2 + len(ds_order_df) * 2, 2):
        if i + 1 >= len(sub_df_sort):  # Boundary check
            break
        site_num = sub_df_sort.iloc[i].Amount
        spot_num = sub_df_sort.iloc[i + 1].Amount
        if site_num + spot_num != 0:
            print("site_num {}".format(i))
            return False

    return True



def verify_v8(sub_df, o2o_order_df, ds_order_df):
    sub_orders = sub_df.Order_id.drop_duplicates().sort_values().tolist()
    raw_orders = o2o_order_df.Order_id.tolist() + ds_order_df.Order_id.tolist()
    return sub_orders == raw_orders

def verify_v10(sub_df):
    sub_amounts = sub_df.Amount.tolist()
    sub_total_amounts = pd.Series(np.cumsum(sub_amounts))
    return len(sub_total_amounts[sub_total_amounts>140]) == 0

def verify_v11(sub_df):
    sub_df_sort = sub_df.sort_values(["Order_id", "Departure"], ascending=[1, 1])
    for i in range(0, len(sub_df_sort) - 1, 2):  # Prevents accessing out-of-bounds index

        pick_addr = sub_df_sort.iloc[i].Addr
        send_addr = sub_df_sort.iloc[i+1].Addr
        if ('S' not in pick_addr) and ('A' not in pick_addr):
            return False
        if ('B' not in send_addr):
            return False
    return True

# Make sure all numeric columns are properly converted to numbers
# This will help prevent type comparison issues
numeric_cols = ['Arrival_time', 'Departure', 'Amount']
for col in numeric_cols:
    if col in sub_df.columns:
        sub_df[col] = pd.to_numeric(sub_df[col], errors='coerce')

v2 = np.array(list(map(lambda arrange: verify2(arrange), arranges)))
if len(v2[v2==False])==0:
    print("pass v2")
else:
    print("V2 FAILED")

v3 = np.array(list(map(lambda arrange: verify3(arrange), arranges)))
if len(v3[v3==False])==0:
    print("pass v3")
else:
    print("V3 FAILED")

if len(sub_df.Order_id) == 2*(len(ds_order_df)+len(o2o_order_df)) and len(sub_df.Order_id.drop_duplicates()) == len(ds_order_df)+len(o2o_order_df):
    print("pass v4")
else:
    print("V4 FAILED")

if verify_v5(sub_df, o2o_order_df, ds_order_df) == True:
    print("pass v5")
else:
    print("V5 FAILED")

if verify_v6(sub_df, o2o_order_df) == True:
    print("pass v6")
else:
    print("V6 FAILED")

if verify_v7(sub_df, o2o_order_df, ds_order_df) == True:
    print("pass v7")
else:
    print("V7 FAILED")

if verify_v8(sub_df, o2o_order_df, ds_order_df) == True:
    print("pass v8")
else:
    print("V8 FAILED")

v9 = np.array(list(map(lambda arrange: verify9(arrange, ds_order_df, o2o_order_df), arranges)))
if len(v9[v9==False])==0:
    print("pass v9")
else:
    print("V9 FAILED")

if verify_v10(sub_df) == True:
    print("pass v10")
else:
    print("V10 FAILED")

if verify_v11(sub_df) == True:
    print("pass v11")
else:
    print("V11 FAILED")

sub_df["Departure"] = pd.to_numeric(sub_df["Departure"], errors='coerce')
result = sum(
    float(arrange.iloc[-1].Departure)
    for arrange in arranges
    if not pd.isna(arrange.iloc[-1].Departure)
)


# total costs
# result = sum(list(map(lambda arrange: arrange.iloc[-1].Departure, arranges)))
print(result)


V2 FAILED
V3 FAILED
V4 FAILED
o2o spot_id 0
V5 FAILED
shop_departure 158
V6 FAILED
pass v7
V8 FAILED
V9 FAILED
pass v10
pass v11
2372390.0739081535


In [ ]:
import math
import random
import pandas as pd
from typing import List
import time
from google.colab import files

START_TIME = 8 * 60
END_TIME = 20 * 60
COURIER_SPEED = 15.0
EARTH_RADIUS = 6378.137

class Location:
    def __init__(self, id, lng, lat, location_type):
        self.id, self.lng, self.lat, self.type = id, lng, lat, location_type

class Order:
    def __init__(self, order_id, spot_id, num_packages):
        self.id, self.spot_id, self.num_packages = order_id, spot_id, num_packages

class EcommerceOrder(Order):
    def __init__(self, order_id, spot_id, site_id, num_packages):
        super().__init__(order_id, spot_id, num_packages)
        self.site_id = site_id

class O2OOrder(Order):
    def __init__(self, order_id, spot_id, shop_id, pickup_time, delivery_time, num_packages):
        super().__init__(order_id, spot_id, num_packages)
        self.shop_id, self.pickup_time, self.delivery_time = shop_id, pickup_time, delivery_time

class CourierRoute:
    def __init__(self, courier_id):
        self.courier_id, self.route, self.current_load = courier_id, [], 0

class DeliveryProblem:
    def __init__(self):
        self.branches, self.pois, self.shops, self.orders, self.couriers, self.distance_cache = {}, {}, {}, [], [], {}

    def calculate_distance(self, loc1, loc2):
        key = tuple(sorted([loc1.id, loc2.id]))
        if key in self.distance_cache:
            return self.distance_cache[key]
        lat1, lng1, lat2, lng2 = map(math.radians, [loc1.lat, loc1.lng, loc2.lat, loc2.lng])
        dlat, dlng = (lat1 - lat2) / 2, (lng1 - lng2) / 2
        distance = 2 * EARTH_RADIUS * math.asin(math.sqrt(math.sin(dlat)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlng)**2))
        self.distance_cache[key] = distance
        return distance

    def calculate_travel_time(self, loc1, loc2):
        return self.calculate_distance(loc1, loc2) / COURIER_SPEED * 60

    def calculate_processing_time(self, num_packages):
        return 3 * math.sqrt(num_packages) + 5

    def convert_time_to_minutes(self, time_str):
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes

def load_problem_data(problem):
    for _, row in pd.read_csv("/content/branches.csv").iterrows():
        problem.branches[row['Site_id']] = Location(row['Site_id'], row['Lng'], row['Lat'], 'branch')
    for _, row in pd.read_csv("/content/pois.csv").iterrows():
        problem.pois[row['Spot_id']] = Location(row['Spot_id'], row['Lng'], row['Lat'], 'poi')
    for _, row in pd.read_csv("/content/shops.csv").iterrows():
        problem.shops[row['Shop_id']] = Location(row['Shop_id'], row['Lng'], row['Lat'], 'shop')
    for _, row in pd.read_csv("/content/ecommerce_orders.csv").iterrows():
        problem.orders.append(EcommerceOrder(row['Order_id'], row['Spot_id'], row['Site_id'], row['Num']))
    for _, row in pd.read_csv("/content/o2o_orders.csv").iterrows():
        pickup = problem.convert_time_to_minutes(row['Pickup_time'])
        delivery = problem.convert_time_to_minutes(row['Delivery_time'])
        problem.orders.append(O2OOrder(row['Order_id'], row['Spot_id'], row['Shop_id'], pickup, delivery, row['Num']))
    for courier_id in pd.read_csv("/content/couriers.csv")['Courier_id']:
        problem.couriers.append(CourierRoute(courier_id))

def create_initial_solution(problem):
    couriers, orders = problem.couriers, problem.orders.copy()
    random.shuffle(orders)
    for i, order in enumerate(orders):
        couriers[i % len(couriers)].route.append(order)
    return couriers

def tabu_search(problem, initial_solution, max_iterations=1000, tabu_size=20):
    best = current = initial_solution
    tabu = []

    def solution_cost(solution):
        total = 0
        for route in solution:
            time, prev = 0, None
            for order in route.route:
                loc = problem.pois.get(order.spot_id) or problem.shops.get(order.spot_id) or problem.branches.get(order.spot_id)
                if prev:
                    time += problem.calculate_travel_time(prev, loc)
                time += problem.calculate_processing_time(order.num_packages)
                prev = loc
            total += time
        return total

    best_cost = solution_cost(best)

    for _ in range(max_iterations):
        neighborhood = []
        for _ in range(10):
            new = [CourierRoute(r.courier_id) for r in current]
            for idx, r in enumerate(current): new[idx].route = r.route.copy()
            r1, r2 = random.sample(range(len(new)), 2)
            if new[r1].route and new[r2].route:
                i1, i2 = random.randint(0, len(new[r1].route)-1), random.randint(0, len(new[r2].route)-1)
                new[r1].route[i1], new[r2].route[i2] = new[r2].route[i2], new[r1].route[i1]
            neighborhood.append(new)

        neighborhood.sort(key=solution_cost)
        current = neighborhood[0]
        current_cost = solution_cost(current)
        if current_cost < best_cost:
            best, best_cost = current, current_cost

        tabu.append(current)
        if len(tabu) > tabu_size: tabu.pop(0)

    return best

problem = DeliveryProblem()
load_problem_data(problem)
initial_solution = create_initial_solution(problem)
final_solution = tabu_search(problem, initial_solution)

solution_data = []
for route in final_solution:
    current_time, current_loc = START_TIME, None

    for order in route.route:
        pickup_loc = problem.branches.get(getattr(order, 'site_id', ''), problem.shops.get(getattr(order, 'shop_id', ''), None))
        delivery_loc = problem.pois[order.spot_id]

        if current_loc:
            current_time += problem.calculate_travel_time(current_loc, pickup_loc)
        arrival_pickup = current_time
        departure_pickup = max(arrival_pickup + problem.calculate_processing_time(order.num_packages), getattr(order, 'pickup_time', arrival_pickup))

        current_time = departure_pickup + problem.calculate_travel_time(pickup_loc, delivery_loc)
        arrival_delivery = current_time
        departure_delivery = max(arrival_delivery + problem.calculate_processing_time(order.num_packages), getattr(order, 'delivery_time', arrival_delivery))

        solution_data.extend([
            {"Courier_id": route.courier_id, "Addr": pickup_loc.id, "Arrival_time": round(arrival_pickup), "Departure": round(departure_pickup), "Amount": order.num_packages, "Order_id": order.id},
            {"Courier_id": route.courier_id, "Addr": delivery_loc.id, "Arrival_time": round(arrival_delivery), "Departure": round(departure_delivery), "Amount": -order.num_packages, "Order_id": order.id}
        ])

        current_loc = delivery_loc
        current_time = departure_delivery

solution_df = pd.DataFrame(solution_data)
solution_df.to_csv('solution_updated.csv', index=False)
files.download('solution_updated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

26/3

In [34]:
import math
import random
import pandas as pd
from typing import List
import time
from google.colab import files

START_TIME = 8 * 60
END_TIME = 20 * 60
COURIER_SPEED = 15.0
EARTH_RADIUS = 6378.137
MAX_PACKAGES = 140
MAX_ORDERS_PER_COURIER = 15  # Debug constraint

class Location:
    def __init__(self, id, lng, lat, location_type):
        self.id, self.lng, self.lat, self.type = id, lng, lat, location_type

class Order:
    def __init__(self, order_id, spot_id, num_packages):
        self.id, self.spot_id, self.num_packages = order_id, spot_id, num_packages

class EcommerceOrder(Order):
    def __init__(self, order_id, spot_id, site_id, num_packages):
        super().__init__(order_id, spot_id, num_packages)
        self.site_id = site_id

class O2OOrder(Order):
    def __init__(self, order_id, spot_id, shop_id, pickup_time, delivery_time, num_packages):
        super().__init__(order_id, spot_id, num_packages)
        self.shop_id, self.pickup_time, self.delivery_time = shop_id, pickup_time, delivery_time

class CourierRoute:
    def __init__(self, courier_id):
        self.courier_id = courier_id
        self.route = []  # list of order ids only

class DeliveryProblem:
    def __init__(self):
        self.branches, self.pois, self.shops = {}, {}, {}
        self.orders, self.order_lookup = [], {}
        self.couriers, self.distance_cache = [], {}

    def calculate_distance(self, loc1, loc2):
        key = tuple(sorted([loc1.id, loc2.id]))
        if key in self.distance_cache:
            return self.distance_cache[key]
        lat1, lng1, lat2, lng2 = map(math.radians, [loc1.lat, loc1.lng, loc2.lat, loc2.lng])
        dlat, dlng = (lat1 - lat2) / 2, (lng1 - lng2) / 2
        distance = 2 * EARTH_RADIUS * math.asin(math.sqrt(math.sin(dlat)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlng)**2))
        self.distance_cache[key] = distance
        return distance

    def calculate_travel_time(self, loc1, loc2):
        return self.calculate_distance(loc1, loc2) / COURIER_SPEED * 60

    def calculate_processing_time(self, num_packages):
        return 3 * math.sqrt(num_packages) + 5

    def convert_time_to_minutes(self, time_str):
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes

def load_problem_data(problem):
    for _, row in pd.read_csv("/content/branches.csv").iterrows():
        problem.branches[row['Site_id']] = Location(row['Site_id'], row['Lng'], row['Lat'], 'branch')
    for _, row in pd.read_csv("/content/pois.csv").iterrows():
        problem.pois[row['Spot_id']] = Location(row['Spot_id'], row['Lng'], row['Lat'], 'poi')
    for _, row in pd.read_csv("/content/shops.csv").iterrows():
        problem.shops[row['Shop_id']] = Location(row['Shop_id'], row['Lng'], row['Lat'], 'shop')
    for _, row in pd.read_csv("/content/ecommerce_orders.csv").iterrows():
        o = EcommerceOrder(row['Order_id'], row['Spot_id'], row['Site_id'], row['Num'])
        problem.orders.append(o)
        problem.order_lookup[o.id] = o
    for _, row in pd.read_csv("/content/o2o_orders.csv").iterrows():
        pickup = problem.convert_time_to_minutes(row['Pickup_time'])
        delivery = problem.convert_time_to_minutes(row['Delivery_time'])
        o = O2OOrder(row['Order_id'], row['Spot_id'], row['Shop_id'], pickup, delivery, row['Num'])
        problem.orders.append(o)
        problem.order_lookup[o.id] = o
    for courier_id in pd.read_csv("/content/couriers.csv")['Courier_id']:
        problem.couriers.append(CourierRoute(courier_id))

def create_initial_solution(problem):
    couriers = problem.couriers
    orders = [o.id for o in problem.orders.copy()]
    random.shuffle(orders)
    for i, order_id in enumerate(orders):
        if len(couriers[i % len(couriers)].route) < MAX_ORDERS_PER_COURIER:
            couriers[i % len(couriers)].route.append(order_id)
    return couriers

def evaluate_route(problem, route):
    current_time = START_TIME
    current_load = 0
    current_loc = None
    for order_id in route.route:
        order = problem.order_lookup[order_id]
        pickup = problem.branches.get(getattr(order, 'site_id', ''), problem.shops.get(getattr(order, 'shop_id', ''), None))
        drop = problem.pois[order.spot_id]

        if current_loc: current_time += problem.calculate_travel_time(current_loc, pickup)
        arrival = current_time
        current_time = max(arrival + problem.calculate_processing_time(order.num_packages), getattr(order, 'pickup_time', arrival))
        if isinstance(order, O2OOrder) and current_time > order.delivery_time:
            return float('inf')
        current_load += order.num_packages
        if current_load > MAX_PACKAGES:
            return float('inf')

        current_time += problem.calculate_travel_time(pickup, drop)
        arrival = current_time
        current_time = max(arrival + problem.calculate_processing_time(order.num_packages), getattr(order, 'delivery_time', arrival))
        if isinstance(order, O2OOrder) and (arrival > order.delivery_time or arrival < order.pickup_time):
            return float('inf')
        if current_time > END_TIME:
            return float('inf')
        current_loc = drop
        current_load -= order.num_packages
    return current_time - START_TIME

def tabu_search(problem, initial_solution, max_iterations=100, tabu_size=10):
    best = current = initial_solution
    best_cost = sum(evaluate_route(problem, r) for r in current)
    tabu = []

    for _ in range(max_iterations):
        neighborhood = []
        for _ in range(5):
            new = [CourierRoute(r.courier_id) for r in current]
            for i, r in enumerate(current):
                new[i].route = r.route.copy()
            r1, r2 = random.sample(range(len(new)), 2)
            if new[r1].route and new[r2].route:
                i1, i2 = random.randint(0, len(new[r1].route)-1), random.randint(0, len(new[r2].route)-1)
                new[r1].route[i1], new[r2].route[i2] = new[r2].route[i2], new[r1].route[i1]
            neighborhood.append(new)

        neighborhood.sort(key=lambda sol: sum(evaluate_route(problem, r) for r in sol))
        candidate = neighborhood[0]
        candidate_cost = sum(evaluate_route(problem, r) for r in candidate)
        if candidate_cost < best_cost:
            best, best_cost = candidate, candidate_cost
        tabu.append(candidate)
        if len(tabu) > tabu_size:
            tabu.pop(0)
    return best

problem = DeliveryProblem()
load_problem_data(problem)
initial_solution = create_initial_solution(problem)
final_solution = tabu_search(problem, initial_solution)

solution_data = []
for route in final_solution:
    current_time = START_TIME
    current_loc = None
    current_load = 0
    for order_id in route.route:
        order = problem.order_lookup[order_id]
        pickup = problem.branches.get(getattr(order, 'site_id', ''), problem.shops.get(getattr(order, 'shop_id', ''), None))
        drop = problem.pois[order.spot_id]

        if current_loc: current_time += problem.calculate_travel_time(current_loc, pickup)
        arrival_pickup = current_time
        departure_pickup = max(arrival_pickup + problem.calculate_processing_time(order.num_packages), getattr(order, 'pickup_time', arrival_pickup))

        current_time = departure_pickup + problem.calculate_travel_time(pickup, drop)
        arrival_drop = current_time
        departure_drop = max(arrival_drop + problem.calculate_processing_time(order.num_packages), getattr(order, 'delivery_time', arrival_drop))

        solution_data.extend([
            {"Courier_id": route.courier_id, "Addr": pickup.id, "Arrival_time": round(arrival_pickup), "Departure": round(departure_pickup), "Amount": order.num_packages, "Order_id": order.id},
            {"Courier_id": route.courier_id, "Addr": drop.id, "Arrival_time": round(arrival_drop), "Departure": round(departure_drop), "Amount": -order.num_packages, "Order_id": order.id}
        ])

        current_time = departure_drop
        current_loc = drop
        current_load = max(0, current_load - order.num_packages)

solution_df = pd.DataFrame(solution_data)
solution_df.to_csv('solution_1.csv', index=False)
files.download('solution_1.csv')

solution_df = pd.DataFrame(solution_data)
solution_df.to_csv('solution_1.csv', index=False)
files.download('solution_1.csv')

# === ✅ Summary Statistics ===
total_couriers_used = sum(1 for r in final_solution if r.route)

print("✅ Total Couriers Used:", total_couriers_used)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Total Couriers Used: 1000


In [16]:
import pandas as pd
import numpy as np

sub_df = pd.read_csv("solution_1.csv")






import pandas as pd
import numpy as np

sub_df = pd.read_csv("solution_1.csv", names=["Courier_id", "Addr", "Arrival_time", "Departure", "Amount", "Order_id"])

SPEED = 15 * 1000.0 / 60
site_df = pd.read_csv("/content/branches.csv")
spot_df = pd.read_csv("/content/pois.csv")
shop_df = pd.read_csv("/content/shops.csv")
ds_order_df = pd.read_csv("/content/ecommerce_orders.csv")
o2o_order_df = pd.read_csv("/content/o2o_orders.csv")
node_lng_dict = {}
node_lat_dict = {}
for i, e in site_df.iterrows():
    node_lng_dict[e.Site_id] = e.Lng
    node_lat_dict[e.Site_id] = e.Lat

for i, e in spot_df.iterrows():
    node_lng_dict[e.Spot_id] = e.Lng
    node_lat_dict[e.Spot_id] = e.Lat

for i, e in shop_df.iterrows():
    node_lng_dict[e.Shop_id] = e.Lng
    node_lat_dict[e.Shop_id] = e.Lat

def time2min(time):
    ts = pd.to_datetime(time, format="%H:%M")
    return ts.hour*60+ts.minute-480

def cal_distance(node1, node2):
    global node_lng_dict, node_lat_dict
    lng1 = node_lng_dict[node1]
    lat1 = node_lat_dict[node1]
    lng2 = node_lng_dict[node2]
    lat2 = node_lat_dict[node2]
    R = 6378137.0
    d_lat = (lat1-lat2)/2.0
    d_lng = (lng1-lng2)/2.0
    S = 2*R*np.arcsin(np.sqrt(np.power(np.sin(np.pi/180*d_lat),2)+np.cos(np.pi/180*lat1)*np.cos(np.pi/180*lat2)*np.power(np.sin(np.pi/180*d_lng),2)))
    return S

def cal_cost(node1, node2):
    global SPEED
    return np.round(cal_distance(node1, node2)/SPEED)

def cal_proctime(x):
    return np.round(3*np.sqrt(x)+5)

arranges = []
for i in range(1, 1001):
    arrange = sub_df[sub_df.Courier_id==("D%04d"%i)]
    if len(arrange)==0:
        continue
    arranges.append(arrange)

def verify2(arrange):
    departure = arrange.iloc[0].Departure
    for i in range(1, len(arrange)):
        arrival_time = arrange.iloc[i].Arrival_time
        if arrival_time < departure:
            return False
        departure = arrange.iloc[i].Departure
    return True

def verify3(arrange):
    for i in range(0, len(arrange)):
        arrival_time = arrange.iloc[i].Arrival_time
        departure = arrange.iloc[i].Departure
        if departure < arrival_time:
            return False
    return True

def verify9(arrange, ds_order_df, o2o_order_df):
    # use the example to verify cal functions
    #mix_path = ["A083","A083","A083","A083","A083","B5800","B7555","B7182","B8307","B8461","A083","A083","A083","B6528","S245","B3266","B3266","B2337","A083","A083","A083","A083","S294","B1940","B6104","B8926","B9072","B6103"]
    #orders = ["F6344","F6360","F6358","F6353","F6354","F6344","F6354","F6353","F6358","F6360","F6349","F6325","F6314","F6349","E0895","E0895","F6325","F6314","F6366","F6345","F6346","F6308","E1088","F6308","F6346","E1088","F6366","F6345"]
    mix_path = arrange.Addr.tolist()
    orders = arrange.Order_id.tolist()
    mix_arrange_df = pd.DataFrame({ "Addr": mix_path, "Order_id": orders})
    mix_arrange_df["Courier_id"] = arrange.iloc[0].Courier_id
    mix_arrange_df["Arrival_time"] = arrange.iloc[0].Arrival_time
    mix_arrange_df["Departure"] = arrange.iloc[0].Departure
    mix_arrange_df["Amount"] = 0
    pre_e = mix_arrange_df.iloc[0]
    pre_addr = pre_e.Addr
    pre_order = pre_e.Order_id
    pre_departure = arrange.iloc[0].Departure
    pre_departure = float(pre_departure)  # Ensure it's a number

    if 'F' in pre_order:
        amount = ds_order_df[ds_order_df["Order_id"]==pre_order].Num.values[0]
    elif 'E' in pre_order:
        amount = o2o_order_df[o2o_order_df["Order_id"]==pre_order].Num.values[0]
    new_mix_arrange_df = pd.DataFrame({"Courier_id": [arrange.iloc[0].Courier_id], "Addr": [pre_addr], "Arrival_time": [arrange.iloc[0].Arrival_time], "Departure": [arrange.iloc[0].Departure], "Amount": [amount],  "Order_id": [pre_order]})
    for i in range(1, len(mix_arrange_df)):
        cur_e = mix_arrange_df.iloc[i]
        cur_addr = cur_e.Addr
        cur_order = cur_e.Order_id
        if 'A' in cur_addr:
            arrival_time = pre_departure + cal_cost(pre_addr, cur_addr)
            amount = ds_order_df[ds_order_df["Order_id"]==cur_order].Num.values[0]
            departure = arrival_time
        elif 'B' in cur_addr and 'F' in cur_order:
            arrival_time = pre_departure + cal_cost(pre_addr, cur_addr)
            amount = -ds_order_df[ds_order_df["Order_id"]==cur_order].Num.values[0]
            departure = arrival_time + cal_proctime(abs(amount))
        elif 'B' in cur_addr and 'E' in cur_order:
            arrival_time = pre_departure + cal_cost(pre_addr, cur_addr)
            amount = -o2o_order_df[o2o_order_df["Order_id"]==cur_order].Num.values[0]
            departure = arrival_time + cal_proctime(abs(amount))
        elif 'S' in cur_addr:
            arrival_time = pre_departure + cal_cost(pre_addr, cur_addr)
            amount = o2o_order_df[o2o_order_df["Order_id"]==cur_order].Num.values[0]
            departure = max(time2min(o2o_order_df[o2o_order_df["Order_id"]==cur_order].Pickup_time.values[0]), arrival_time)
        new_row = pd.DataFrame({"Courier_id": [arrange.iloc[0].Courier_id], "Addr": [cur_addr], "Arrival_time": [arrival_time], "Departure": [departure], "Amount": [amount],  "Order_id": [cur_order]})
        new_mix_arrange_df = pd.concat([new_mix_arrange_df, new_row], ignore_index=True)
        cmp_e = arrange.iloc[i]
        if cur_addr != cmp_e.Addr or amount != cmp_e.Amount or arrival_time != cmp_e.Arrival_time or departure != cmp_e.Departure:
            # print("arrange {}".format(i))
            return False
        pre_addr = cur_addr
        pre_departure = departure
    return True


def verify_v5(sub_df, o2o_order_df, ds_order_df):
    sub_df_sort = sub_df.sort_values(["Order_id", "Departure"], ascending=[1, 1])
    for i in range(0, len(o2o_order_df)*2, 2):
        shop_id = sub_df_sort.iloc[i].Addr
        shop_amount = sub_df_sort.iloc[i].Amount
        spot_id = sub_df_sort.iloc[i+1].Addr
        spot_amount = sub_df_sort.iloc[i+1].Amount
        if shop_id != o2o_order_df.iloc[i//2].Shop_id:
            print("o2o shop_id {}".format(i))
            return False
        if spot_id != o2o_order_df.iloc[i//2].Spot_id:
            print("o2o spot_id {}".format(i))
            return False
        if shop_amount != o2o_order_df.iloc[i//2].Num:
            print("o2o shop_amount {}".format(i))
            return False
        if spot_amount != -o2o_order_df.iloc[i//2].Num:
            print("o2o spot_amount {}".format(i))
            return False
    for i in range(len(o2o_order_df)*2, len(o2o_order_df)*2+len(ds_order_df)*2, 2):
        site_id = sub_df_sort.iloc[i].Addr
        site_amount = sub_df_sort.iloc[i].Amount
        spot_id = sub_df_sort.iloc[i+1].Addr
        spot_amount = sub_df_sort.iloc[i+1].Amount
        if site_id != ds_order_df.iloc[i//2-len(o2o_order_df)].Site_id:
            print("ds site_id {}".format(i))
            return False
        if spot_id != ds_order_df.iloc[i//2-len(o2o_order_df)].Spot_id:
            print("ds spot_id {}".format(i))
            return False
        if site_amount != ds_order_df.iloc[i//2-len(o2o_order_df)].Num:
            print("ds site_amount {}".format(i))
            return False
        if spot_amount != -ds_order_df.iloc[i//2-len(o2o_order_df)].Num:
            print("ds spot_amount {}".format(i))
            return False
    return True

def verify_v6(sub_df, o2o_order_df):
    sub_df_sort = sub_df.sort_values(["Order_id", "Departure"], ascending=[1, 1])
    for i in range(0, len(o2o_order_df)*2, 2):
        shop_departure = sub_df_sort.iloc[i].Departure
        # Convert shop_departure to int if it's a string
        if isinstance(shop_departure, str):
            try:
                shop_departure = int(shop_departure)
            except ValueError:
                print(f"Warning: Could not convert shop_departure '{shop_departure}' to integer")
                return False

        pickup_time = o2o_order_df.iloc[i//2].Pickup_time
        pickup_min = time2min(pickup_time)

        if shop_departure < pickup_min:
            print("shop_departure {}".format(i))
            return False
    return True

def verify_v7(sub_df, o2o_order_df, ds_order_df):
    sub_df_sort = sub_df.sort_values(["Order_id", "Departure"], ascending=[1, 1])
    for i in range(0, len(o2o_order_df) * 2, 2):
        if i + 1 >= len(sub_df_sort):  # Boundary check
            break
        shop_num = sub_df_sort.iloc[i].Amount
        spot_num = sub_df_sort.iloc[i + 1].Amount
        if shop_num + spot_num != 0:
            print("shop_num {}".format(i))
            return False

    for i in range(len(o2o_order_df) * 2, len(o2o_order_df) * 2 + len(ds_order_df) * 2, 2):
        if i + 1 >= len(sub_df_sort):  # Boundary check
            break
        site_num = sub_df_sort.iloc[i].Amount
        spot_num = sub_df_sort.iloc[i + 1].Amount
        if site_num + spot_num != 0:
            print("site_num {}".format(i))
            return False

    return True



def verify_v8(sub_df, o2o_order_df, ds_order_df):
    sub_orders = sub_df.Order_id.drop_duplicates().sort_values().tolist()
    raw_orders = o2o_order_df.Order_id.tolist() + ds_order_df.Order_id.tolist()
    return sub_orders == raw_orders

def verify_v10(sub_df):
    sub_amounts = sub_df.Amount.tolist()
    sub_total_amounts = pd.Series(np.cumsum(sub_amounts))
    return len(sub_total_amounts[sub_total_amounts>140]) == 0

def verify_v11(sub_df):
    sub_df_sort = sub_df.sort_values(["Order_id", "Departure"], ascending=[1, 1])
    for i in range(0, len(sub_df_sort) - 1, 2):  # Prevents accessing out-of-bounds index

        pick_addr = sub_df_sort.iloc[i].Addr
        send_addr = sub_df_sort.iloc[i+1].Addr
        if ('S' not in pick_addr) and ('A' not in pick_addr):
            return False
        if ('B' not in send_addr):
            return False
    return True

# Make sure all numeric columns are properly converted to numbers
# This will help prevent type comparison issues
numeric_cols = ['Arrival_time', 'Departure', 'Amount']
for col in numeric_cols:
    if col in sub_df.columns:
        sub_df[col] = pd.to_numeric(sub_df[col], errors='coerce')

v2 = np.array(list(map(lambda arrange: verify2(arrange), arranges)))
if len(v2[v2==False])==0:
    print("pass v2")
else:
    print("V2 FAILED")

v3 = np.array(list(map(lambda arrange: verify3(arrange), arranges)))
if len(v3[v3==False])==0:
    print("pass v3")
else:
    print("V3 FAILED")

if len(sub_df.Order_id) == 2*(len(ds_order_df)+len(o2o_order_df)) and len(sub_df.Order_id.drop_duplicates()) == len(ds_order_df)+len(o2o_order_df):
    print("pass v4")
else:
    print("V4 FAILED")

if verify_v5(sub_df, o2o_order_df, ds_order_df) == True:
    print("pass v5")
else:
    print("V5 FAILED")

if verify_v6(sub_df, o2o_order_df) == True:
    print("pass v6")
else:
    print("V6 FAILED")

if verify_v7(sub_df, o2o_order_df, ds_order_df) == True:
    print("pass v7")
else:
    print("V7 FAILED")

if verify_v8(sub_df, o2o_order_df, ds_order_df) == True:
    print("pass v8")
else:
    print("V8 FAILED")

v9 = np.array(list(map(lambda arrange: verify9(arrange, ds_order_df, o2o_order_df), arranges)))
if len(v9[v9==False])==0:
    print("pass v9")
else:
    print("V9 FAILED")

if verify_v10(sub_df) == True:
    print("pass v10")
else:
    print("V10 FAILED")

if verify_v11(sub_df) == True:
    print("pass v11")
else:
    print("V11 FAILED")

sub_df["Departure"] = pd.to_numeric(sub_df["Departure"], errors='coerce')
result = sum(
    float(arrange.iloc[-1].Departure)
    for arrange in arranges
    if not pd.isna(arrange.iloc[-1].Departure)
)


# total costs
# result = sum(list(map(lambda arrange: arrange.iloc[-1].Departure, arranges)))
print(result)


V2 FAILED
V3 FAILED
V4 FAILED
o2o spot_id 0
V5 FAILED
pass v6
pass v7
V8 FAILED
V9 FAILED
pass v10
pass v11
2744010.0
